In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
file = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/rs1356432@gmail.com/googleplaystore.csv")

In [0]:
file.show(2)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|              Genres|    Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|      2.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+-----------+------------+
only showing top 2 rows



In [0]:
file = file.drop("Size","Content Rating","Current Ver", "Android Ver")

In [0]:
file.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+----------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|    Last Updated|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+----------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design| January 7, 2018|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|January 15, 2018|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+----------------+
only showing top 2 rows



In [0]:
file.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)



In [0]:
from pyspark.sql.functions import regexp_replace,col
file=file.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
    .withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
        .withColumn("Installs",col("Installs").cast(IntegerType()))\
            .withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
                .withColumn("Price",col("Price").cast(IntegerType()))

In [0]:
file.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)



In [0]:
file.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|   Last Updated|
+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|January 7, 2018|
+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
only showing top 1 row



In [0]:
file = file.withColumn("Rating",col("Rating").cast(DoubleType()))

In [0]:
file.createOrReplaceTempView("mydata")

In [0]:
%sql select * from mydata
limit 5

App,Category,Rating,Reviews,Installs,Type,Price,Genres,Last Updated
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,10000,Free,0,Art & Design,"January 7, 2018"
Coloring book moana,ART_AND_DESIGN,3.9,967,500000,Free,0,Art & Design;Pretend Play,"January 15, 2018"
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,5000000,Free,0,Art & Design,"August 1, 2018"
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,50000000,Free,0,Art & Design,"June 8, 2018"
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,100000,Free,0,Art & Design;Creativity,"June 20, 2018"


In [0]:
%sql select App, sum(Reviews) from mydata
group by 1
order by 2 desc
limit 5

App,sum(Reviews)
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958


Databricks visualization. Run in Databricks to view.

In [0]:
file.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|   Last Updated|
+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|January 7, 2018|
+--------------------+--------------+------+-------+--------+----+-----+------------+---------------+
only showing top 1 row



In [0]:
%sql select App, Type  ,sum(Installs) from mydata
group by 1,2
order by 3 desc
limit 5

App,Type,sum(Installs)
Subway Surfers,Free,6000000000
Instagram,Free,4000000000
Google Drive,Free,4000000000
Hangouts,Free,4000000000
Google Photos,Free,4000000000


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select App, sum(Reviews) as Total_Reviews from mydata
group by 1
order by 2 desc
limit 5

App,Total_Reviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958


In [0]:
%sql select App , Type , sum(Price) from mydata 
group by 1,2
order by 3 desc
limit 4 

App,Type,sum(Price)
I'm Rich - Trump Edition,Paid,400
I am Rich,Paid,399
I am Rich Plus,Paid,399
I AM RICH PRO PLUS,Paid,399


Databricks visualization. Run in Databricks to view.